# Chap3. wor2vec
## 3.4 CBOW 모델 구현

앞 절에서 CBOW모델의 학습을 위해 데이터를 준비했었다. CBOW 모델 학습을 위해 데이터를 다시 불러보자!

In [1]:
import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

contexts, target = create_contexts_target(corpus, window_size=1)

vocab_size = len(word_to_id)

# contexts와 target을 one-hot vector로 변환
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

In [14]:
print('target: ', target)
print('contexts: ', contexts)

# 원-핫 베겉로 target과 context데이터가 생성된 것을 확인할 수 있다.

target:  [[0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0]]
contexts:  [[[1 0 0 0 0 0 0]
  [0 0 1 0 0 0 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 1 0 0 0]]

 [[0 0 1 0 0 0 0]
  [0 0 0 0 1 0 0]]

 [[0 0 0 1 0 0 0]
  [0 1 0 0 0 0 0]]

 [[0 0 0 0 1 0 0]
  [0 0 0 0 0 1 0]]

 [[0 1 0 0 0 0 0]
  [0 0 0 0 0 0 1]]]


In [24]:
contexts.shape

(6, 2, 7)

In [18]:
contexts[:, 0]  

array([[1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0]])

In [22]:
print(contexts[0, 0])
print(contexts[1, 0])
print(contexts[2, 0])
print(contexts[3, 0])
print(contexts[4, 0])
print(contexts[5, 0])

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 1 0 0 0 0 0]


헷갈릴때는 이렇게 print를 찍고, 어느부분에서 혼란이 오는지 파악하는 것도 하나의 방법일까?

In [17]:
contexts[:, 0].shape

(6, 7)

In [13]:
contexts[:, 1]

array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1]])

SimpleCBOW라는 이름으로 CBOW모델을 구현해보자.

In [23]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):  # 초기화 메서드의 인수로 vocab_size(어휘 수)와 은닉층의 뉴런 수(hidden_size)를 받는다.
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')  # 무작위 Weight 초기화, 타입은 float 32비트로 만들어준다.
        W_out = 0.01 * np.random.randn(H, V).astype('f')  # W_in (V, H), W_out (H, V)
        
        # 계층 생성
        self.in_layer0 = MatMul(W_in)  # 입력값과 가중치값의 곱을 구해준다.
        self.in_layer1 = MatMul(W_in)  # 입력에서 사용하는 MatMul 계층의 수는 입력에 사용되는 단어의 수만큼 만들어야 한다.
                                       # 윈도우의 크기가 2이므로 2개의 MatMul계층을 만들어준다.
                                       # 윈도우의 크기가 n이라면 n개의 MatMul계층을 만들어주는 것이다.
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()
        
        # 모든 가중치와 기울기를 리스트에 모아준다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []  # 신경망에서 사용되는 매개변수와 기울기를 인스턴스 변수인 params와 grads 리스트에 각각 모아둔다.
        
        for layer in layers:  
            self.params += layer.params
            self.grads += layer.grads        
        
        # 인스턴스 변수에 단어의 분산 표현을 저장
        self.word_vecs = W_in
        
    # 순전파 forward 메서드 구현 
    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])  # contexts는 3차원 배열인데...
                                                     # contexts[:, 0]
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss
    
    # 역전파
    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)  # descent_softmax
        da = self.out_layer.backward(ds)     # descent_matmul
        da *= 0.5
        self.in_layer0.backward(da)
        self.in_layer1.backward(da)
        return None

코드를 그저 따라쳤던게 아닌가 다시한번 냉정하게 생각해보자...<br>
내가 지금 구현하고 있는 모델이 어떤 구조를 가지고 있으며 해당 부분들이 코드로 어떻게 구현되는지 제대로 알고 넘어갈 수 있도록 하자.

📌 Check Point<br>
CBOW 모델의 신경망 구성을 다시 한번 체크하기<br>
신경망을 코드로 구현하는 과정 꼼꼼하게 짚고 넘어가기

### 3.4.1 학습 코드 구현
학습을 수행 하는데 있어서 Trainer 클래스를 사용한다.<br>
"1장에서 설명한 Trainer 클래스를 사용한다."<br>
안타깝지만... 기억속에 없다...

In [ ]:
import sys
sys.path.append('..')
from common.trainer import Trainer
from common.optimizer import Adam


📌 &lt;To-do&gt;<br>

20.03.25.Wed. pm 3:12
<br>
* common trainer
* common optimizer
* SimpleCBOW.py

In [7]:
contexts.shape

(6, 2, 7)

In [6]:
contexts[:, 0].shape

(6, 7)

In [13]:
import numpy as np

a = np.array([0, 1, 2, 3, 4])
a

array([0, 1, 2, 3, 4])

In [16]:
a.shape

(5,)

In [21]:
a.shape[0]

5

In [23]:
a.ndim

1

In [19]:
one_hot = np.zeros((5, 5), dtype=np.int32)
one_hot

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [20]:
one_hot.shape

(5, 5)